# Evaluation of traffic light detection 
based on [CarND Object Detection Lab](https://github.com/udacity/CarND-Object-Detection-Lab)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageDraw
from PIL import ImageColor
import time
import os
import xml.etree.cElementTree as ET

%matplotlib inline
plt.style.use('ggplot')

## Helper functions

In [2]:
# Colors (one for each class)
cmap = ImageColor.colormap
# print("Number of colors =", len(cmap))
COLOR_LIST = sorted([c for c in cmap.keys()])

#
# Utility funcs
#

def filter_boxes(min_score, boxes, scores, classes):
    """Return boxes with a confidence >= `min_score`"""
    n = len(classes)
    idxs = []
    for i in range(n):
        if scores[i] >= min_score:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    return filtered_boxes, filtered_scores, filtered_classes

def to_image_coords(boxes, height, width):
    """
    The original box coordinate output is normalized, i.e [0, 1].
    
    This converts it back to the original coordinate based on the image
    size.
    """
    box_coords = np.zeros_like(boxes)
    box_coords[:, 0] = boxes[:, 0] * height
    box_coords[:, 1] = boxes[:, 1] * width
    box_coords[:, 2] = boxes[:, 2] * height
    box_coords[:, 3] = boxes[:, 3] * width
    
    return box_coords

def draw_boxes(image, boxes, classes, thickness=4):
    """Draw bounding boxes on the image"""
    draw = ImageDraw.Draw(image)
    for i in range(len(boxes)):
        bot, left, top, right = boxes[i, ...]
        class_id = int(classes[i])
        color = COLOR_LIST[class_id]
        draw.line([(left, top), (left, bot), (right, bot), (right, top), (left, top)], width=thickness, fill=color)
        
def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph

In [3]:
# Frozen inference graph files. NOTE: change the path to where you saved the models.
base_dir = 'frozen_models'
FASTER_RCNN_GRAPH_FILE = os.path.join(base_dir, 'real', 'faster_rcnn_resnet101_coco_2018_01_28_ck_1368_1096', 'frozen_inference_graph.pb')

In [4]:
detection_graph = load_graph(FASTER_RCNN_GRAPH_FILE)      
category_index = {1: {'id': 1, 'name': 'green'}, 2: {'id': 2, 'name': 'red'}, 3: {'id': 3, 'name': 'yellow'}, 4: {'id': 4, 'name': 'off'}}


def inference_test(detection_graph, image):
    image_np = np.expand_dims(np.asarray(image, dtype=np.uint8), 0)
    
    # The input placeholder for the image.
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

    with tf.Session(graph=detection_graph) as sess:                
        # Actual detection.
        (boxes, scores, classes) = sess.run([detection_boxes, detection_scores, detection_classes], 
                                            feed_dict={image_tensor: image_np})

        # Remove unnecessary dimensions
        boxes = np.squeeze(boxes)
        scores = np.squeeze(scores)
        classes = np.squeeze(classes)

        confidence_cutoff = 0.8
        # Filter boxes with a confidence score less than `confidence_cutoff`
        boxes, scores, classes = filter_boxes(confidence_cutoff, boxes, scores, classes)
    
    return boxes, scores, classes

In [5]:
test_set_path = 'data/real_data/test'

image_files = []
xml_files = []
for file_name in os.listdir(test_set_path):
    if file_name.endswith(".jpg"):
        image_files.append(file_name)
    elif file_name.endswith(".xml"):
        xml_files.append(file_name)

image_files = sorted(image_files)
xml_files = sorted(xml_files)

In [10]:
result_strings = []
truth_list = []
prediction_list = []

num_correct = 0
for i in range(len(image_files)):
    image = Image.open(os.path.join(test_set_path, image_files[i]))
    
    boxes, scores, classes = inference_test(detection_graph, image)

    width, height = image.size
    box_coords = to_image_coords(boxes, height, width)
    
    truth = -1
    try:
        root = ET.ElementTree(file=os.path.join(test_set_path, xml_files[i])).getroot()
        truth = root.find('object').find('name').text
    except:
        truth = 'no_box'
    
    prediction = 'no_box'
    class_index = -1
    score = -1
    if len(boxes) > 0:
        prediction = category_index[classes[0]]['name']
        class_index = classes[0]
        score = scores[0]
        
    num_boxes = len(boxes)

    #file_name, truth, prediction, class_index, score, num_boxes
    result = '{}, {}, {}, {}, {}, {}'.format(image_files[i], truth, prediction, class_index, score, num_boxes)
    
    result_strings.append(result)
    truth_list.append(truth)
    prediction_list.append(prediction)
    
    if (truth == prediction):
        num_correct += 1
    
    print(result)
    
   

left0000.jpg, green, green, 1.0, 0.999753534794, 1
left0040.jpg, green, green, 1.0, 0.999578773975, 1
left0041.jpg, green, green, 1.0, 0.999535083771, 1
left0042.jpg, green, green, 1.0, 0.999387979507, 1
left0050.jpg, green, green, 1.0, 0.999617576599, 1
left0051.jpg, green, green, 1.0, 0.999639749527, 1
left0052.jpg, green, green, 1.0, 0.999558389187, 1
left0060.jpg, green, green, 1.0, 0.99946397543, 1
left0061.jpg, green, green, 1.0, 0.999517679214, 1
left0062.jpg, green, green, 1.0, 0.998994767666, 1
left0070.jpg, green, green, 1.0, 0.999001204967, 1
left0071.jpg, green, green, 1.0, 0.999364554882, 1
left0072.jpg, green, green, 1.0, 0.99893862009, 1
left0080.jpg, yellow, yellow, 3.0, 0.999903202057, 1
left0081.jpg, yellow, yellow, 3.0, 0.999878883362, 1
left0082.jpg, yellow, yellow, 3.0, 0.999911427498, 1
left0090.jpg, yellow, yellow, 3.0, 0.999558508396, 1
left0091.jpg, yellow, yellow, 3.0, 0.999876022339, 1
left0092.jpg, yellow, yellow, 3.0, 0.999767959118, 1
left0100.jpg, yellow,

In [11]:
print(num_correct)

129
